## Импорты

In [23]:
import pandas as pd
from matplotlib import pyplot as plt
from pyvis.network import Network
import networkx as nx

CatmaidInstance

In [24]:
import pymaid

catmaid_url = 'https://l1em.catmaid.virtualflybrain.org'
http_user = None
http_password = None
project_id = 1

rm = pymaid.CatmaidInstance(catmaid_url, http_user, http_password, project_id)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


## Класс для получения и сохранения структуры одного нейрона

In [25]:
class simplified_structure:
    def __init__(self, neuron:pymaid.CatmaidNeuronList):
        self.neuron:pymaid.CatmaidNeuronList = neuron
        self.nodes:nx.MultiDiGraph = None
        # названия в формате id_обекта, если знак id положителен, то это нода скелета, если отрицателен то это коннектор
        self.build_structure()
        self.add_connectors_to_graph()
        self.simplify_directed_graph()

    def build_structure(self):
        nodes = self.neuron.nodes
        graph = nx.MultiDiGraph()
        for idt, parent, ntype in zip(nodes['node_id'], nodes['parent_id'], nodes['type']):
            # сома это root
            if (a:=idt >= 0):
                graph.add_node(idt, type = ntype) 
            if (b:=parent >= 0):
                graph.add_node(parent, type = ntype)
            if a and b:
                graph.add_edge(idt, parent, nodes_inside = [])
        self.nodes = graph

    def add_connectors_to_graph(self):
        post_connectors = pymaid.get_connectors(self.neuron, 'postsynaptic_to')
        for idt in post_connectors['connector_id']:
            cID = -idt
            q = self.neuron.connectors[self.neuron.connectors['connector_id'] == idt]
            self.nodes.add_node(cID, type = 'Postsynaptic')
            for node_id in q['node_id']:
                self.nodes.add_edge(cID, node_id, nodes_inside = [])

        pre_connectors = pymaid.get_connectors(self.neuron, 'presynaptic_to')
        for idt in pre_connectors['connector_id']:
            cID = -idt
            q = self.neuron.connectors[self.neuron.connectors['connector_id'] == idt]
            self.nodes.add_node(cID, type = 'Presynaptic')
            for node_id in q['node_id']:
                self.nodes.add_edge(node_id, cID, nodes_inside = [])

    def simplify_directed_graph(self):
        def keep_nodes(graph, vid):
            return graph.nodes(True)[vid]['type'] == 'root' or vid < 0
        G = self.nodes
        original = len(self.nodes)
        while True:
            # Находим все вершины с in-degree=1 и out-degree=1
            nodes_to_remove = [
                node for node in G.nodes() 
                if G.in_degree(node) == 1 and G.out_degree(node) == 1 and not keep_nodes(G, node)
            ]

            if not nodes_to_remove:
                break # Если таких вершин нет, завершаем

            for node in nodes_to_remove:
                # Если узел уже был удален на предыдущей итерации этого же цикла, пропускаем
                if node not in G: 
                    continue

                # Получаем единственного предшественника и преемника
                # NetworkX гарантирует, что list(predecessors/successors) вернет один элемент,
                # если степень равна 1.
                u = list(G.predecessors(node))
                v = list(G.successors(node))

                if len(u) != 1 or len(v) != 1:
                    raise Exception('Этот эксепшен не должен никогда вызватся, но он вызвался и значит что то пошло не так')
                u = u[0]
                v = v[0]

                nodes_inside = sum((edge[-1]['nodes_inside'] for edge in G.edges(node, data = True)), [])
                    
                if u != v:
                    G.add_edge(u, v, nodes_inside = [node] + nodes_inside)

                G.remove_node(node)

        after = len(self.nodes)
        print('removed', original - after, 'nodes.', f'Efficiency: {round(100*(1 - after/original), 1)}%')

    def save_as_nx_graph(self, path):
        nx.write_gml(self.nodes, path)

    def save_as_pyvis_html(self, path):
        net = Network(notebook = False, directed = True)
        net.from_nx(self.nodes)
        for node in net.nodes:
            node['label'] = node['type']
            if node['type'] == 'root':
                node['size'] = 30
            if node['id'] < 0:
                node['color'] = 'orange'
                node['size'] = 5

        net.show_buttons(filter_=['physics'])
        net.save_graph(path)

## Тестируем simplified_structure

In [26]:
import os

class composed_network:
    def __init__(self, paths):
        self.paths = paths
        self.graphs = {}
        for path in paths:
            self.graphs[path] = nx.read_gml(path)
            for node_id, attr_dict in self.graphs[path].nodes(True):
                filename = os.path.basename(path)
                attr_dict['owner'] = filename

        self.combined_graph = nx.compose_all(self.graphs.values())

    def save_as_gml(self, path):
        nx.write_gml(self.combined_graph, path)

    def save_as_pyvis_html(self, path, colors:dict = None):
        net = Network(notebook=False, directed=True)
        net.from_nx(self.combined_graph)
        net.show_buttons(filter_=['physics'])
        for node in net.nodes:
            if node['type'] == 'root':
                node['size'] = 30
            if int(node['id']) < 0:
                node['shape'] = 'square'
                node['size'] = 5
                node['color'] = 'orange'
                if node['type'] not in ('Postsynaptic', 'Presynaptic'):
                    node['label'] = node['type']
                else:
                    node['label'] = None
            else:
                node['label'] = node['type']
                    
        if colors:
            for node in net.nodes:
                if int(node['id']) > 0:
                    if node['owner'] in colors:
                        node['color'] = colors[node['owner']]
        net.save_graph(path)

In [27]:
A = pymaid.get_neuron(9469519)
B = pymaid.get_neuron(29)

In [28]:
#S = simplified_structure(B)
#S.save_as_pyvis_html(f'visualizaions/test/graph_of_{B.id}.html')

## Класс для объединения структур нескольких нейронов

In [29]:
a = simplified_structure(A)
b = simplified_structure(B)

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 3612 nodes. Efficiency: 76.2%


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 2442 nodes. Efficiency: 75.7%


In [30]:
a.save_as_nx_graph(f'./neurons/{a.neuron.id}.gml')
b.save_as_nx_graph(f'./neurons/{b.neuron.id}.gml')

In [31]:

s = composed_network([f'./neurons/{b.neuron.id}.gml', f'./neurons/{a.neuron.id}.gml'])
s.save_as_gml('./combined.gml')
s.save_as_pyvis_html(f"./neurons/visual/ns{a.neuron.id}_and_{b.neuron.id}.html", colors = {
    str(a.neuron.id) + '.gml': 'blue',
    str(b.neuron.id) + '.gml': 'green',
})

In [32]:
ids = ['7055857', '1805418', '14260575', '5835799', '10160250', '7840203', '5019924', '13986477', '10167078', '7982896', '4119387', '17591442', '4227544', '10495502', '8069478', '3913629', '11279244', '16846805', '8980589', '3664102']
paths = []
for ind in ids:
    A = pymaid.get_neuron(int(ind))
    path = "./del_this/neurons/" + ind + ".gml"
    Q = simplified_structure(A)
    Q.save_as_nx_graph(path)
    paths.append(path)

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 2872 nodes. Efficiency: 65.9%


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 2021 nodes. Efficiency: 81.3%


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


...

removed 4153 nodes. Efficiency: 67.3%


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 6870 nodes. Efficiency: 67.9%


INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


removed 3824 nodes. Efficiency: 74.4%


In [33]:
S = composed_network(paths)
S.save_as_gml("./del_this/cs_ns_combined.gml")